### Why Data Security is Important?

![](Images/93/93 Data Security.jpg)

### Databricks Security Features

![](Images/93/93 Databricks Security Features.jpg)

### Create Sample Delta Table

In [0]:
from delta.tables import *
DeltaTable.create(spark) \
  .tableName('dimEmployee') \
  .addColumn('empID', 'INT') \
  .addColumn('empName', 'STRING') \
  .addColumn('SSN', 'STRING') \
  .execute()

### Insert PII Data

In [0]:
%sql
insert into dimEmployee values(100, 'Mike', '2345671');
insert into dimEmployee values(200, 'David', '5632178');
insert into dimEmployee values(300, 'Peter', '1456782');

### View Data

In [0]:
%sql
select * from dimEmployee

### Cryptography Library Installation

In [0]:
pip install cryptography

### Generate Encryption/Decryption Key

In [0]:
from cryptography.fernet import Fernet

key = Fernet.generate_key()
fernet = Fernet(key)

### Encrypt Sample Data

In [0]:
PIIData = b"testmail@gmail.com"
PIIencoded = fernet.encrypt(PIIData)
print(PIIencoded)


### Decrypt Sample Data

In [0]:
print(fernet.decrypt(PIIencoded))

### Define UDF to Encypt Data

In [0]:
def encrypt_data(data, KEY):
    from cryptography.fernet import Fernet
    fernet = Fernet(KEY)
    dataB = bytes(data, 'utf-8')
    encrypted_data = fernet.encrypt(dataB)
    encrypted_data = str(encrypted_data.decode('ASCII'))
    return encrypted_data

### Define UDF to Decrypt Data

In [0]:
def decrypt_data(encrypted_data, KEY):
    from cryptography.fernet import Fernet
    fernet = Fernet(KEY)
    decrpted_data = fernet.decrypt(encrypted_data.encode()).decode()
    return decrpted_data

### Register UDF's

In [0]:
from pyspark.sql.functions import udf, lit, md5
from pyspark.sql.types import StringType

encryptin = udf(encrypt_data, StringType())
decryptin = udf(decrypt_data, StringType())

### Encrypt the Data

In [0]:
df = spark.table('dimemployee')
encryptedDF = df.withColumn('SSN_Encrypted', encryptin('SSN', lit(key)))
display(encryptedDF)

### Decrypt the Data

In [0]:
decryptedDF = encryptedDF.withColumn('SSN_Decrypted', decryptin('SSN_Encrypted', lit(key)))
display(decryptedDF)